# Preprocessing TP1
Aca se escriben los distintos preprocesados y se describe rapidamente su funcionalidad


In [33]:
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib
import seaborn as sns
import numpy as np
np.warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score
import category_encoders as ce

sns.set()

df_data = pd.read_csv('https://drive.google.com/uc?export=download&id=1i-KJ2lSvM7OQH0Yd59bX01VoZcq8Sglq')
df_decision = pd.read_csv('https://drive.google.com/uc?export=download&id=1km-AEIMnWVGqMtK-W28n59hqS5Kufhd0')

In [34]:
columnsTitles=["tipo_de_sala","id_usuario","genero","edad","amigos",
               "parientes","precio_ticket",'nombre_sede','cant_acompañantes','volveria']

In [35]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

Este preprossing esta basado en lo visto y hecho en el TP1, y es nuestro primero approach de preprossing

In [36]:
def preprod_tp1(df_datos, df_predict):
    df = pd.merge(df_datos, df_predict, how='inner', left_on='id_usuario', right_on='id_usuario')
    df = df.drop(['id_ticket', 'fila', 'nombre'], axis=1)
    df['cant_acompañantes'] = df['parientes'] + df['amigos']
    df = df.reindex(columns=columnsTitles)
    return df

Reemplazo los valores nulos de edad con la moda, la media o la meediana, segun que quiera

In [37]:
def replace_nulls_edad(df, metrica):
    s = df['edad']
    if metrica == 'moda':
        df = df.replace({'edad': np.nan}, s.mode())
    elif metrica == 'mediana':
        df = df.replace({'edad': np.nan}, s.median())
    elif metrica == 'media':
        df = df.replace({'edad': np.nan}, s.mean())
    df = df.round({'edad': 1})
    return df

Encodeo todos los atributos categoricos

In [38]:
def encodear_atributos_categoricos(df):
    encoder = ce.BinaryEncoder(cols=['tipo_de_sala', 'genero', 'nombre_sede'],return_df=True)
    df = encoder.fit_transform(df) 
    return df

Agarro todos los atributos numericos y los normalizo

In [39]:
def normalizar_atributos_numericos(df):
    columnas_numericas = df.select_dtypes(include=numerics).columns.to_list()
    maximo = df[columnas_numericas].max()
    minimo = df[columnas_numericas].min()
    df[columnas_numericas] = (df[columnas_numericas] - df[columnas_numericas].min()) / (df[columnas_numericas].max() - df[columnas_numericas].min())
    return df

In [40]:
#df.dtypes

In [41]:
df_decision.head()

,id_usuario,volveria
0,117,0
1,658,0
2,794,0
3,455,0
4,173,1


In [42]:
df = preprod_tp1(df_data, df_decision)
df = replace_nulls_edad(df, 'media')
df.head()

,tipo_de_sala,id_usuario,genero,edad,amigos,parientes,precio_ticket,nombre_sede,cant_acompañantes,volveria
0,4d,117,hombre,73.5,0,0,1,fiumark_quilmes,0,0
1,4d,658,mujer,35.0,1,1,2,fiumark_quilmes,2,0
2,normal,794,hombre,32.7,0,0,3,fiumark_chacarita,0,0
3,4d,455,hombre,32.7,0,0,1,fiumark_palermo,0,0
4,4d,173,mujer,4.0,1,1,2,fiumark_palermo,2,1


In [43]:
df = normalizar_atributos_numericos(df)
df.head()

,tipo_de_sala,id_usuario,genero,edad,amigos,parientes,precio_ticket,nombre_sede,cant_acompañantes,volveria
0,4d,0.130337,hombre,0.880653,0.000,0.000000,0.000000,fiumark_quilmes,0.0,0.0
1,4d,0.738202,mujer,0.396985,0.125,0.166667,0.020408,fiumark_quilmes,0.2,0.0
2,normal,0.891011,hombre,0.368090,0.000,0.000000,0.040816,fiumark_chacarita,0.0,0.0
3,4d,0.510112,hombre,0.368090,0.000,0.000000,0.000000,fiumark_palermo,0.0,0.0
4,4d,0.193258,mujer,0.007538,0.125,0.166667,0.020408,fiumark_palermo,0.2,1.0


In [44]:
columnas_numericas = df.select_dtypes(include=numerics).columns.to_list()
df[columnas_numericas].max()

id_usuario           1.0
edad                 1.0
amigos               1.0
parientes            1.0
precio_ticket        1.0
cant_acompañantes    1.0
volveria             1.0
dtype: float64

In [45]:
df.tipo_de_sala.value_counts()

4d        447
normal    187
3d        167
Name: tipo_de_sala, dtype: int64

In [46]:
df.tipo_de_sala.value_counts()

4d        447
normal    187
3d        167
Name: tipo_de_sala, dtype: int64

In [47]:
df = encodear_atributos_categoricos(df)
df.head()

,tipo_de_sala_0,tipo_de_sala_1,tipo_de_sala_2,id_usuario,genero_0,genero_1,edad,amigos,parientes,precio_ticket,nombre_sede_0,nombre_sede_1,nombre_sede_2,cant_acompañantes,volveria
0,0,0,1,0.130337,0,1,0.880653,0.000,0.000000,0.000000,0,0,1,0.0,0.0
1,0,0,1,0.738202,1,0,0.396985,0.125,0.166667,0.020408,0,0,1,0.2,0.0
2,0,1,0,0.891011,0,1,0.368090,0.000,0.000000,0.040816,0,1,0,0.0,0.0
3,0,0,1,0.510112,0,1,0.368090,0.000,0.000000,0.000000,0,1,1,0.0,0.0
4,0,0,1,0.193258,1,0,0.007538,0.125,0.166667,0.020408,0,1,1,0.2,1.0


In [48]:
df.shape

(801, 15)